# Exploring Kickstarter Project Data
## by Michael Mosin

## Preliminary Wrangling

This document explores a dataset comprised of various attributes for an assortment of 3786 Kickstarter projects

In [ ]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

In [ ]:
# Import dataset
# Dataset downloaded from CSV link under "2019-05-16" on site: https://webrobots.io/kickstarter-datasets/
df = pd.read_csv('Kickstarter.csv')

In [ ]:
# Adding ability to view all dataframe columns
# as per https://stackoverflow.com/questions/49188960/how-to-show-all-of-columns-name-on-pandas-dataframe/49189503
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
df.info()

In [ ]:
df.duplicated().value_counts()

In [ ]:
# Make copy of main dataframe so as to keep original data intact.
df_copy = df.copy()
df_copy.shape

## Tracking Data Quality and Tidiness Issues:

### Quality:

- Variables "created_at", "deadline", "launched_at", and "state_changed_at" are set in unix time instead of readable datetime
- Variables with financial values such as "converted_pledged_amount", "goal", "pledged", and "usd_pledged" are set to different decimal places, and should be rounded to at most two decimal places
- Variables "friends", "is_backing", "is_starred", and "permissions" only have one entry and should be dropped
- Only two entries are missing data for "location" (not a big deal, given that we have "country" data; these)
- Only eleven entries are missing data for "usd_type" (this variable is not important to the investigation)

### Tidiness:

- Data entries in the columns "category", "creator", "location", "photo", "profile", and "urls" contain multiple pieces of information. If separated, they could be their own dataframes or made into additional columns in the main dataframe.
    - The "category" variable can garner category and sub-category info for the projects
    - The "location" variable can garner data regarding the project's state name, city name, and city type
    - The "creator","photo", "profile", "urls" variables have no data that is relevant to this project and should be dropped


## Addressing Data Quality and Tidiness Issues

### Quality: 

#### Remove (essentially) empty columns

In [ ]:
# Remove variables that only have one entry.
df_copy = df_copy.drop(columns = ["friends", "is_backing", "is_starred", "permissions"])
df_copy.shape

#### Fix time categories

In [ ]:
# Converting unix time to readable date-time
# as per https://stackoverflow.com/questions/19231871/convert-unix-time-to-readable-date-in-pandas-dataframe
date_cols = ["created_at", "deadline", "launched_at", "state_changed_at"]
for i in date_cols:
    df_copy[i] = pd.to_datetime(df_copy[i],unit='s')

df_copy[date_cols].head()

In [ ]:
df_copy[date_cols].describe()

#### Fix financial categories

In [ ]:
# Round financial values to at most two decimal places
money_cols = ["converted_pledged_amount", "goal", "pledged", "usd_pledged"]
for i in money_cols:
    df_copy = df_copy.round(2)

df_copy[money_cols].head()

### Tidiness: 

#### Feature Engineering - address tidiness issue of "category" variable by creating variables holding extracted values for main category and sub-category:

In [ ]:
# View full string entry for "category" variable of fifth row entry to gauge the complexity of category strings:
df_copy['category'][4]

In [ ]:
# Extract product categories and sub-catgories from strings in "category" variable into their own columns in dataframe
# (Used regular expression)

import re  

df_copy['main_cat'] = ''
df_copy['sub_cat'] = ''

for i in np.arange(df_copy.shape[0]):
    match = re.findall('(([ &]|\w+)+)', df_copy['category'][i])
    df_copy['main_cat'][i] = match[5][0].title()
    df_copy['sub_cat'][i] = match[6][0].title()
df_copy[['name','main_cat', 'sub_cat', 'category']].head() 

In [ ]:
df_copy.main_cat.value_counts()

In [ ]:
df_copy.sub_cat.value_counts()

#### Feature Engineering - address tidiness issue of "location" variable by creating variables holding extracted values for state, city, and type of city:

In [ ]:
df_copy[df_copy.location.isnull()]

In [ ]:
# View full string entry for "location" variable of third row entry to gauge the complexity of location strings:
df_copy['location'][2]

In [ ]:
# Extract product (country) states, cities, and city types from strings in "location" variable into their own columns in dataframe
# (Used regular expression)

df_copy['location_state'] = ''
df_copy['location_city'] = ''
df_copy['location_type'] = ''

for i in np.arange(df_copy.shape[0]):
    if pd.notna(df_copy.location[i]) == True:
        match = re.findall('((?:[^"]\w+)+)', df_copy['location'][i])
        df_copy['location_state'][i] = match[17]
        df_copy['location_city'][i] = match[3]
        df_copy['location_type'][i] = match[19]
    else:
        df_copy['location_state'][i] = 'NaN'
        df_copy['location_city'][i] = 'NaN'
        df_copy['location_type'][i] = 'NaN'
    

In [ ]:
df_copy[['name', 'country', 'location_state', 'location_city', 'location_type', 'location']][1930:1933]

### What is the structure of your dataset?

There are 3786 Kickstarter projects in this dataset, with a total of 37 features, some of which are untidy, and some of which are not of interest for my exploration. I have engineered a few categorical features (related to project ctegories and location) which may come to be useful for exploration. 


### What is/are the main feature(s) of interest in your dataset?

I am interested in finding out which project qualities correlate with different types of project outcomes (or, the final "state" of the project). 


### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

I believe the following features could illuminate patterns in project outcomes:
    - Number of backers
    - Length of time project was open
    - Total money pledged (relative to funding goal)
    - Project category
    - Project location (country, city type)
    - If project had "spotlight"
    - If project was "staff pick"

In [ ]:
df_copy.to_csv('data_wrangled.csv', index=False)

## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.

> Make sure that, after every plot or related series of plots, that you
include a Markdown cell with comments about what you observed, and what
you plan on investigating next.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!